# Data

In [63]:
import pandas as pd
url1 = r"../activa/data/hol_listado_empresas.xlsx"
url2 = r"../activa/data/Activa_listado.xlsx"
df1 = pd.read_excel(url1)
df2 = pd.read_excel(url2,sheet_name=1)

# Modules & functions

In [64]:
def cleanString(x:str):
    x = x.split(" ")
    rt = []
    for i in x:
        if len(i)<=4:
            continue
        else:
            rt.append(i)

    return "".join(rt).strip()
def str_clean(x):
    data = []
    index = 0

    for i in x:
        if i == "_" or i== "-":
            continue
        else:
            data.append(i)
    return "".join(data)
def getNumber(x:str):
    x = x.strip()
    x = x.split()
    return x[-1]
import re
def digitStract(x:str):
    try:
        return re.findall(r'\d+', x)[0]
    except:
        return "0000"

In [65]:
from geoCodeOSM import (
    geoCodeLat,
    geoCodeLon,
    geoCodeLatLon
)

# Last Date-Auth

In [66]:
df2["UniqueId"] = df2["DIRECCIÓN"].map(digitStract) + "-"+ df2["CUIT"].astype(str)

df1["cuitempresa"] = df1["cuitempresa"].map(str_clean)
df1.columns = ['id_tramite', 'FechaHabAprobada', 'Calle', 'Numero', 'Barrio',
       'FechaVencimientoHab', 'CUIT', 'razonsocial', 'NombreFantasia',
       'RubroNombre']
df1["UniqueId"] = df1["Numero"].map(digitStract) + "-"+ df1["CUIT"].astype(str)



df1["CUIT"] = df1["CUIT"].map(str_clean)
df2["CUIT"] = df2["CUIT"].map(lambda x: str(x)).map(str_clean)


# obtain last date in authorization date

df1["location-ID"] = df1["Calle"] + " " + df1["Numero"].astype(str) + " " + df1["CUIT"].astype(str)
df1["FechaVencimientoHab"] = df1["FechaVencimientoHab"].astype("datetime64[D]")
df1["last-date"] = 0

locationDate = {}

for i in df1["location-ID"].unique():
    subset = df1[df1["location-ID"] == i]
    if subset["FechaVencimientoHab"].max():
        locationDate[i] = subset["FechaVencimientoHab"].max()
    else:
        locationDate[i] = subset["FechaVencimientoHab"][0]
locationDate

for i in range(len(df1)):
    r = df1["location-ID"][i]
    if r in locationDate.keys():
        df1["last-date"][i] = locationDate[r]

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10892\1890085460.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["last-date"][i] = locationDate[r]


# Add Lat-Lon 

In [67]:
df1["Address"] = df1["Calle"] + " " + df1["Numero"].astype(str) + ", " + "Córdoba," + " " + "Argentina"
df1["LatLon"] = df1["Address"].map(geoCodeLatLon)

IndexError: list index out of range

# Merge

In [ ]:
dfCuit = pd.merge(df2, df1, on='CUIT')
#dfCuit = dfCuit.drop_duplicates(subset=['id_tramite'])

dfCuit["validUniqueId"] = dfCuit["UniqueId_x"] == dfCuit["UniqueId_y"]

# Using Google GeoCode

In [ ]:
"""
import pandas as pd
url1 = r"../activa/data/hol_listado_empresas.xlsx"
df1 = pd.read_excel(url1)

df1["Address"] = df1["Calle"] + " " + df1["Numero"].astype(str) + ", " + "Córdoba," + " " + "Argentina"
df1 = df1.drop_duplicates(subset=['Address'])
df1.to_excel(r"../activa/data/toGeo.xlsx")

len(df1)

#900 calls in one day // no factibe

"""

32268